# Carbon Properties Simulation Tutorial

This notebook provides a step-by-step guide for simulating and analyzing key properties of carbon (graphene/diamond) using GPUMD. It covers lattice constant, elastic constants, density of states (DOS), and quenching processes.

## 1. Objective

- Calculate the lattice constant and elastic constants of carbon.
- Compute the phonon density of states (DOS) using velocity auto-correlation.
- Demonstrate quenching simulation.
        

## 2. Required Dependencies

- Python 3.x
- numpy
- matplotlib
- ASE (Atomic Simulation Environment)
- GPUMD (for MD simulation)
markdown
markdown
## 8. Diffusion Coefficient Calculation

Diffusion coefficient is calculated using the `compute_sdc` command in GPUMD. The output file `sdc.out` contains the time evolution of the self-diffusion coefficient at different temperatures.
code
python
# Example: Plot diffusion coefficient at 5000K
sdc_data = np.loadtxt('D_Diffusion_Coefficient/5000K/sdc.out')
time = sdc_data[:,0]
sdc = sdc_data[:,1:4].mean(axis=1)
plt.plot(time, sdc)
plt.xlabel('Time (ps)')
plt.ylabel('Self-Diffusion Coefficient (Å²/ps)')
plt.title('Diffusion Coefficient at 5000K')
plt.show()
markdown
markdown
## 9. Thermal Conductivity Calculation

Thermal conductivity is calculated using the HNEMD and SHC methods in GPUMD. The output file `shc_results.txt` contains the running thermal conductivity.
code
python
# Example: Plot thermal conductivity
shc_data = np.loadtxt('F_Thermal_Conductivity/shc_results.txt')
time = shc_data[:,0]
kappa = shc_data[:,1]
plt.plot(time, kappa)
plt.xlabel('Time (ns)')
plt.ylabel('Thermal Conductivity (W/mK)')
plt.title('Thermal Conductivity vs. Time')
plt.show()
markdown
markdown
## 10. Uniaxial Tension Simulation

Uniaxial tension is performed using the `deform` command in GPUMD. The stress-strain curve can be analyzed from the output files.
code
python
# Example: Plot stress-strain curve
# Replace 'stress_strain.out' with the actual output file name
if os.path.exists('G_Uniaxial_Tension/Run/stress_strain.out'):
    ss_data = np.loadtxt('G_Uniaxial_Tension/Run/stress_strain.out')
    strain = ss_data[:,0]
    stress = ss_data[:,1]
    plt.plot(strain, stress)
    plt.xlabel('Strain')
    plt.ylabel('Stress (GPa)')
    plt.title('Stress-Strain Curve (Uniaxial Tension)')
    plt.show()
else:
    print('stress_strain.out not found. Please run GPUMD uniaxial tension simulation.')

Install Python packages with:
```python
!pip install numpy matplotlib ase
```
GPUMD: [https://gpumd.org/](https://gpumd.org/)

## 3. Lattice Constant Calculation

The lattice constant is calculated by running GPUMD with the following input file (`run.in`):

```plaintext
potential         ../../../potentials/nep/C_2022_NEP3.txt
compute_cohesive  0.98 1.03 51
```

This scans the lattice constant from 0.98 to 1.03 times the reference value in 51 steps.

In [ ]:
# Example: Visualize cohesive energy vs. lattice constant
import numpy as np
import matplotlib.pyplot as plt
data = np.loadtxt('A_lattice_constant/cohesive.out')  # Columns: a, E_cohesive
plt.plot(data[:,0], data[:,1], marker='o')
plt.xlabel('Lattice Constant (Å)')
plt.ylabel('Cohesive Energy (eV/atom)')
plt.title('Cohesive Energy vs. Lattice Constant')
plt.grid(True)
plt.show()

## 4. Elastic Constants Calculation

Elastic constants are calculated using the following input file (`run.in`):

```plaintext
potential  ../../../potentials/nep/C_2022_NEP3.txt
compute_elastic  0.01 cubic
```

This applies small strains and computes the elastic constants for cubic symmetry.

In [ ]:
# Example: Read and display elastic constants
elastic = np.loadtxt('B_elastic_constants/elastic.out')
print('Elastic constants (GPa):')
print(elastic)

## 5. Phonon Density of States (DOS) Calculation

DOS is calculated from the velocity auto-correlation function (VAC) using GPUMD.
The workflow is:
- Generate atomic structure (e.g., graphene nanoribbon) with ASE.
- Prepare GPUMD input files.
- Run GPUMD to obtain `dos.out` and `mvac.out`.
        

In [ ]:
from ase.build import graphene_nanoribbon
from ase.io import write
gnr = graphene_nanoribbon(60, 36, type='armchair', sheet=True, vacuum=3.35/2, C_C=1.44)
gnr.euler_rotate(theta=90)
l = gnr.cell.lengths()
gnr.cell = gnr.cell.new((l[0], l[2], l[1]))
l = l[2]
gnr.center()
gnr.pbc = [True, True, False]
write('model.xyz', gnr)

## 6. Quenching Simulation

Quenching can be performed by running GPUMD with a temperature ramp in the input file.
        

In [ ]:
# Example: Visualize temperature evolution during quenching
quench_data = np.loadtxt('C_quenching/thermo.out')
plt.plot(quench_data[:,0], quench_data[:,1])
plt.xlabel('Time (fs)')
plt.ylabel('Temperature (K)')
plt.title('Temperature Evolution During Quenching')
plt.show()

## 7. Results Visualization: DOS and VAC

Load and plot the DOS and VAC results from GPUMD output files.

In [ ]:
dos_array = np.loadtxt('D_DOS/dos.out')
dos = {}
dos['omega'], dos['DOSx'], dos['DOSy'], dos['DOSz'] = dos_array[:,0], dos_array[:,1], dos_array[:,2], dos_array[:,3]
dos['DOSxyz'] = dos['DOSx'] + dos['DOSy'] + dos['DOSz']
dos['nu'] = dos['omega'] / (2 * np.pi)
vac_array = np.loadtxt('D_DOS/mvac.out')
vac = {}
vac['t'], vac['VACx'], vac['VACy'], vac['VACz'] = vac_array[:,0], vac_array[:,1], vac_array[:,2], vac_array[:,3]
vac['VACxyz'] = vac['VACx'] + vac['VACy'] + vac['VACz']
vac['VACxyz'] /= vac['VACxyz'].max()
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(vac['t'], vac['VACxyz'])
plt.xlabel('Correlation Time (ps)')
plt.ylabel('VAC (Normalized)')
plt.title('Velocity Auto-Correlation')
plt.subplot(1,2,2)
plt.plot(dos['nu'], dos['DOSxyz'])
plt.xlabel('Frequency (THz)')
plt.ylabel('DOS (1/THz)')
plt.title('Phonon Density of States')
plt.tight_layout()
plt.show()

## 8. Notes on NEP Training

If you need to train a NEP model, use the NEP package and your training dataset. Example training command:
```bash
nep train --input training_data.extxyz --output C_2022_NEP3.txt
```
        

## 9. References
- Zheyong Fan et al., [GPUMD: A package for constructing accurate machine-learned potentials and performing highly efficient atomistic simulations](https://doi.org/10.1063/5.0106617), J. Chem. Phys. 157, 114801 (2022).
- GPUMD Documentation: https://gpumd.org/
- NEP: https://github.com/zhyan0603/NEP